# Chapter 3: Keras and Data Retrieval in TensorFlow 2

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch03-Keras-and-Data-Retrieval/3.2.Creating_Input_Pipelines.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries and some setup

In [1]:
import os
import requests
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import tensorflow_datasets as tfds

def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
fix_random_seed(4321)
print("TensorFlow version: {}".format(tf.__version__))

data_dir = 'data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)


TensorFlow version: 2.5.0


## Using the `tf.data` API to retrieve data

Here we will be using the `tf.data` API to feed a dataset containing images of flowers. The dataset has a folder containing the images and a CSV file listing filenames and their corresponding label as an integer. We will write a TensorFlow data pipeline that does the following.

* Extract filenames and classes from the CSV
* Read in the images from the extracted filenames and resize them to 64x64
* Convert the class labels to one-hot encoded vectors
* Combine the processed images and one-hot encoded vectors to a single dataset
* Finally, shuffle the data and output as batches

### Downloading the data
The dataset is available at https://www.kaggle.com/olgabelitskaya/flower-color-images/data . 

You need to download the zip file available in this URL and place it in the `data` folder in the `Ch02` folder. You need to select the `flower_images` folder and click download which will download it as a zip.

In [19]:
# Section 3.2

import os
from zipfile import ZipFile

# Extracting the flowers image data to a directory
if os.path.exists('data/flower_images.zip'):
    zfile = ZipFile('data/flower_images.zip')
    zfile.extractall('data')
else:
    print("Did you download the dataset as a zip file and place it in the Ch02/data folder? Make sure you download flower_images.zip from https://www.kaggle.com/olgabelitskaya/flower-color-images/data")

## Creating a tf.data.Dataset 

Here we are creating the `tf.data` pipeline that executes the above steps.

In [20]:
# Section 3.2
# Code listing 3.5

import tensorflow as tf
import os
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Read the CSV file with TensorFlow
# The os.path.sep at the end is important for the get_image function
data_dir = os.path.join('data','flower_images') + os.path.sep
assert os.path.exists(data_dir)
csv_ds = tf.data.experimental.CsvDataset(
    os.path.join(data_dir,'flower_labels.csv') , ("",-1), header=True
)
# Separate the image names and labels to two separate sets
fname_ds = csv_ds.map(lambda a,b: a)
label_ds = csv_ds.map(lambda a,b: b)

def get_image(file_path):
    
    img = tf.io.read_file(data_dir + file_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [64, 64])

# Get the images by running get_image across all the filenames
image_ds = fname_ds.map(get_image)
print("The image dataset contains: {}".format(image_ds))
# Create onehot encoded labels from label data
label_ds = label_ds.map(lambda x: tf.one_hot(x, depth=10))
# Zip the images and labels together
data_ds = tf.data.Dataset.zip((image_ds, label_ds))

# Shuffle the data so that we get a mix of labels in every batch
data_ds = data_ds.shuffle(buffer_size= 20)
# Define a batch of size 5 
data_ds = data_ds.batch(5)
# Iterate through the data to see what it contains
for item in data_ds:
    print(item)
    break


The image dataset contains: <MapDataset shapes: (64, 64, 3), types: tf.float32>
(<tf.Tensor: shape=(5, 64, 64, 3), dtype=float32, numpy=
array([[[[0.33823532, 0.5235294 , 0.7205882 ],
         [0.3421569 , 0.5254902 , 0.72156864],
         [0.34117648, 0.5245098 , 0.72450984],
         ...,
         [0.34313726, 0.52745104, 0.72745097],
         [0.34117648, 0.5254902 , 0.7264706 ],
         [0.34411764, 0.5284314 , 0.7254902 ]],

        [[0.34117648, 0.5245098 , 0.7264706 ],
         [0.34019607, 0.52745104, 0.72156864],
         [0.3392157 , 0.5235294 , 0.7254902 ],
         ...,
         [0.34509805, 0.5284314 , 0.7294118 ],
         [0.34411764, 0.5284314 , 0.7294118 ],
         [0.34313726, 0.5284314 , 0.72843134]],

        [[0.34313726, 0.52647066, 0.72156864],
         [0.34117648, 0.5284314 , 0.72352946],
         [0.34411764, 0.5254902 , 0.7235294 ],
         ...,
         [0.34411764, 0.5284314 , 0.7294118 ],
         [0.34509805, 0.5294118 , 0.7294118 ],
         [0.346078

### Defining and training a model

Here we are defining a simple Convolution Neural Network (CNN) model to train it on the image data we just retrieved. You don't have to worry about the technical details of CNNs right now. We will discuss them in detail in the next chapter.

In [21]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a Convolution neural network for you to train for the flowers data
# We will discuss convolution neural networks in more detail later
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(64,64,3)),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Training the model with the tf.data pipeline
model.fit(data_ds, epochs=10)

Epoch 1/10
42/42 [==============================] - 1s 8ms/step - loss: 3.0899 - acc: 0.2190
Epoch 2/10
42/42 [==============================] - 0s 3ms/step - loss: 1.3278 - acc: 0.5476
Epoch 3/10
42/42 [==============================] - 0s 3ms/step - loss: 0.5898 - acc: 0.8286
Epoch 4/10
42/42 [==============================] - 0s 3ms/step - loss: 0.2633 - acc: 0.9286
Epoch 5/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0751 - acc: 0.9762
Epoch 6/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0161 - acc: 1.0000
Epoch 7/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0060 - acc: 1.0000
Epoch 8/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0025 - acc: 1.0000
Epoch 9/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0020 - acc: 1.0000
Epoch 10/10
42/42 [==============================] - 0s 3ms/step - loss: 0.0016 - acc: 1.0000


## Using Keras data generators to retrieve data

Instead of `tf.data` API let us use the Keras `ImageDataGenerator` to retrieve the data. As you can see, the `ImageDataGenerator` involves much less code than the using the `tf.data` API. 

In [22]:
# Section 3.2
# Code listing 3.6

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

data_dir = os.path.join('data','flower_images')

# Defining an image data generator provided in Keras
img_gen = ImageDataGenerator()

# Reading the CSV files containing filenames and labels
labels_df = pd.read_csv(os.path.join(data_dir, 'flower_labels.csv'), header=0)

# Generating data using the flow_from_dataframe function
gen_iter = img_gen.flow_from_dataframe(
    dataframe=labels_df, directory=data_dir, x_col='file', y_col='label', class_mode='raw', batch_size=5, target_size=(64,64))

# Iterating through the data
for item in gen_iter:
    print(item)
    break

Found 210 validated image filenames.
(array([[[[ 20.,  25.,  16.],
         [ 28.,  38.,  24.],
         [ 18.,  20.,  13.],
         ...,
         [ 14.,  17.,  12.],
         [  9.,  11.,   8.],
         [ 10.,  12.,  11.]],

        [[ 30.,  41.,  25.],
         [ 16.,  21.,  13.],
         [ 23.,  31.,  16.],
         ...,
         [ 16.,  19.,  13.],
         [ 12.,  13.,  10.],
         [  8.,   9.,   8.]],

        [[ 21.,  29.,  17.],
         [ 23.,  31.,  18.],
         [ 23.,  32.,  17.],
         ...,
         [ 14.,  16.,  12.],
         [ 12.,  14.,  10.],
         [ 12.,  14.,  10.]],

        ...,

        [[ 31.,  45.,  21.],
         [ 27.,  41.,  17.],
         [ 19.,  23.,  14.],
         ...,
         [ 25.,  33.,  21.],
         [ 16.,  19.,  13.],
         [ 16.,  20.,  13.]],

        [[ 29.,  44.,  19.],
         [ 31.,  46.,  22.],
         [ 32.,  44.,  19.],
         ...,
         [ 20.,  27.,  16.],
         [ 13.,  18.,  12.],
         [ 17.,  21.,  15.]],

## Using the `tensorflow-datasets` library

Here we will use the `tensorflow-datasets` package. It is a curated list of popular datasets available for machine learning projects. With this package you can download a dataset in a single line. This means you don't have to worry about downloading/extracting/formatting data manually. All of that will be already done when you import data using the `tensorflow-datasets` library.

### Lists the available datasets

In [23]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow as tf
# See all registered datasets
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'booksum',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19',
 'covid19sum',
 'crema_d',
 'cs_restaurants',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'd4rl_adroit_door',
 'd4rl_adr

### Download the Cifar10 dataset and view information

In [24]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar10", with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='cifar10',
    full_name='cifar10/3.0.2',
    description="""
    The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_path='C:\\Users\\thush\\tensorflow_datasets\\cifar10\\3.0.2',
    download_size=162.17 MiB,
    dataset_size=132.40 MiB,
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=50000, num_shards=1>,
    },
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning

### Exploring the data 

Here we will print the `data` and see what it provides. Then we will need to batch the data as data is provided as individual samples when you import it from `tensorflow-datasets`.

In [25]:
print(data)

{'train': <PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>, 'test': <PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>}


In [26]:
# Print some training data
train_ds = data["train"].batch(16)
for item in train_ds:
    print(item)
    break

{'id': <tf.Tensor: shape=(16,), dtype=string, numpy=
array([b'train_16399', b'train_01680', b'train_47917', b'train_17307',
       b'train_27051', b'train_48736', b'train_26263', b'train_01456',
       b'train_19135', b'train_31598', b'train_12970', b'train_04223',
       b'train_27152', b'train_49635', b'train_04093', b'train_17537'],
      dtype=object)>, 'image': <tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 

In [27]:
# Section 3.2

import tensorflow as tf

# Defining a dataset with batch size 16
train_ds = data["train"].batch(16)

# Creating a dataset that returns an (image, one-hot label) tuple
def format_data(x):
    return (x["image"], tf.one_hot(x["label"], depth=10))
train_ds = train_ds.map(format_data)

# Iterating the dataset
for item in train_ds:
    print(item)
    break

(<tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 123],
         [179, 142, 121],
         ...,
         [198, 163, 132],
         [201, 166, 135],
         [207, 174, 143]],

        [[187, 150, 117],
         [181, 143, 115],
         [175, 136, 113],
         ...,
         [201, 164, 132],
         [205, 168, 135],
         [207, 171, 139]],

        [[195, 161, 126],
         [187, 153, 123],
         [186, 151

### Training a simple CNN on the Cifar10 data

In [28]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a simple convolution neural network to process the CIFAR data
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(32,32,3)),
    Flatten(),
    Dense(10, activation='softmax')
])
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Fitting the model on the data for 25 epochs
model.fit(train_ds, epochs=25)

Epoch 1/25
3125/3125 [==============================] - 6s 2ms/step - loss: 3.3836 - acc: 0.1169
Epoch 2/25
3125/3125 [==============================] - 5s 1ms/step - loss: 2.2725 - acc: 0.1394
Epoch 3/25
3125/3125 [==============================] - 5s 1ms/step - loss: 2.2407 - acc: 0.1473
Epoch 4/25
3125/3125 [==============================] - 5s 1ms/step - loss: 2.2080 - acc: 0.1574
Epoch 5/25
3125/3125 [==============================] - 5s 1ms/step - loss: 2.1793 - acc: 0.1702
Epoch 6/25
3125/3125 [==============================] - 5s 2ms/step - loss: 2.1701 - acc: 0.1734
Epoch 7/25
3125/3125 [==============================] - 5s 1ms/step - loss: 2.1364 - acc: 0.1855
Epoch 8/25
3125/3125 [==============================] - 5s 2ms/step - loss: 2.1252 - acc: 0.1929
Epoch 9/25
3125/3125 [==============================] - 5s 2ms/step - loss: 2.0978 - acc: 0.2013
Epoch 10/25
3125/3125 [==============================] - 5s 2ms/step - loss: 2.0793 - acc: 0.2134
Epoch 11/25
3125/3125 [======